In [ ]:
# Install necessary libraries
!pip install pyspark pandas numpy seaborn matplotlib

distutils: c:\users\user\appdata\local\programs\python\python37\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python37\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\user\appdata\local\programs\python\python37\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python37\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


#Spark Prep

In [ ]:
# Check pyspark installation by building spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("Colab")\
        .config('spark.ui.port', '4050').getOrCreate()

spark

#Data Preprocessing

In [ ]:
# Import the pandas library
import pandas as pd

# Read the data from the dataset
df = spark.read.csv("../Dataset/zomato.csv", header=True, inferSchema=True)

store_data = pd.read_csv("../Dataset/zomato.csv", dtype='unicode')

In [ ]:
df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------+--------------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|                 url|             address|                name|        online_order|          book_table| rate|               votes|         phone|            location|          rest_type|          dish_liked|            cuisines|approx_cost(for two people)|        reviews_list|           menu_item|     listed_in(type)|     listed_in(city)|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------+--------------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+-----------

In [ ]:
data = store_data[["online_order", "book_table", "rest_type", "rate"]]
data.head()
data = data.rename(columns={'rate':'grade'})
frame = data.copy()
frame['grade']=frame['grade'].str.replace('/5','')
def is_number(s):
    try:
        return float(s)
    except ValueError:
        return 'NaN'
frame.grade = frame.grade.apply(lambda x: is_number(x) )
frame.dropna(subset = ["online_order", "book_table", "rest_type", "grade"], inplace=True)

frame.grade = frame.grade.astype(float)

# applying dropna again since some fields were filled with a '-'
frame.dropna(subset = ["online_order", "book_table", "rest_type",  "grade"], inplace=True)

def convert_grade(grade):
    
      if grade <=5.0 and grade > 4.0:
        grade = 'Grade A'
      elif grade <=4.0 and grade > 3.0:
        grade = 'Grade B'
      elif grade <=3.0 and grade > 2.0:
        grade = 'Grade C'
      elif grade <=2.0 and grade > 1.0:
        grade = 'Grade D'
      else :
        grade = 'Grade E'
      
      return grade

frame.grade = frame.grade.apply(lambda x: convert_grade(x) )

frame.online_order = frame.online_order.astype(str)
frame.book_table = frame.book_table.astype(str)
frame.rest_type = frame.rest_type.astype(str)
frame.grade = frame.grade.astype(str)

In [ ]:
split_frame = frame.assign(rest_type=frame['rest_type'].str.split(',')).explode('rest_type')

In [ ]:
df = spark.createDataFrame(split_frame)
df.show()

Py4JJavaError: An error occurred while calling o59.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 7) (DESKTOP-0VK3SSI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [ ]:
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol="online_order", outputCol = "online_order_index"),
 StringIndexer(inputCol="book_table", outputCol = "book_table_index"),
 StringIndexer(inputCol="rest_type", outputCol = "rest_type_index"),
 StringIndexer(inputCol="grade", outputCol = "label")]


In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=indexers)
#Fitting a model to the input dataset.
indexed_df = pipeline.fit(df).transform(df)
indexed_df.show(5,False) 

In [ ]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ["online_order_index", "book_table_index", "rest_type_index"],outputCol = "features")
vindexed_df = vectorAssembler.transform(indexed_df)
vindexed_df.show(5, False)

In [ ]:
splits = vindexed_df.randomSplit([0.6,0.4], 42) 
train_df = splits[0]
test_df = splits[1]

In [ ]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(modelType="multinomial")
# train the model
nbmodel = nb.fit(train_df)
# select example rows to display.
predictions_df = nbmodel.transform(test_df)
predictions_df.show(5, True)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorAccuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorAccuracy.evaluate(predictions_df)

evaluatorf1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions_df)

evaluatorPrecision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluatorPrecision.evaluate(predictions_df)

evaluatorRecall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluatorRecall.evaluate(predictions_df)

print("Test set accuracy = " + str(accuracy))
print("Test set f1 measure = " + str(f1))
print("Test set precision = " + str(precision))
print("Test set recall = " + str(recall))